In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/kiolpujy/class4.csv
/kaggle/input/kiolpujy/class3.csv
/kaggle/input/kiolpujy/class1.csv
/kaggle/input/kiolpujy/class6.csv
/kaggle/input/kiolpujy/class0.csv
/kaggle/input/kiolpujy/class5.csv
/kaggle/input/kiolpujy/class2.csv


In [2]:
df = pd.read_csv('/kaggle/input/kiolpujy/class1.csv')
df.columns

Index(['id', 'pixels', 'class_1'], dtype='object')

In [3]:
pip install wtfml


Note: you may need to restart the kernel to use updated packages.


In [4]:
import os
import torch
import albumentations

import numpy as np
import pandas as pd

import torch.nn as nn
from sklearn import metrics
from sklearn import model_selection
from torch.nn import functional as F

from wtfml.utils import EarlyStopping
from wtfml.engine import Engine

/opt/conda/lib/python3.10/site-packages/albumentations/__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.15 (you have 1.4.14). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [5]:
import torch.nn as nn
import torch.utils.model_zoo as model_zoo


__all__ = ['ResNet', 'resnet18', 'resnet34', 'resnet50', 'resnet101',
           'resnet152']


model_urls = {
    'resnet18': 'https://download.pytorch.org/models/resnet18-5c106cde.pth',
    'resnet34': 'https://download.pytorch.org/models/resnet34-333f7ec4.pth',
    'resnet50': 'https://download.pytorch.org/models/resnet50-19c8e357.pth',
    'resnet101': 'https://download.pytorch.org/models/resnet101-5d3b4d8f.pth',
    'resnet152': 'https://download.pytorch.org/models/resnet152-b121ed2d.pth',
}


def conv3x3(in_planes, out_planes, stride=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=1, bias=False)


def conv1x1(in_planes, out_planes, stride=1):
    """1x1 convolution"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = nn.BatchNorm2d(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(Bottleneck, self).__init__()
        self.conv1 = conv1x1(inplanes, planes)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = conv3x3(planes, planes, stride)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = conv1x1(planes, planes * self.expansion)
        self.bn3 = nn.BatchNorm2d(planes * self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out


class ResNet(nn.Module):

    def __init__(self, block, layers, num_classes=1000, zero_init_residual=False):
        super(ResNet, self).__init__()
        self.inplanes = 64
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)
        self.loss_fn = nn.CrossEntropyLoss()

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        # Zero-initialize the last BN in each residual branch,
        # so that the residual branch starts with zeros, and each residual block behaves like an identity.
        # This improves the model by 0.2~0.3% according to https://arxiv.org/abs/1706.02677
        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, Bottleneck):
                    nn.init.constant_(m.bn3.weight, 0)
                elif isinstance(m, BasicBlock):
                    nn.init.constant_(m.bn2.weight, 0)

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    def forward(self, x,targets = None):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        
        # To save pre-final layer's features

        feat = self.avgpool(self.layer4(x))
        feat = feat.view(feat.size(0), -1)
        x = self.fc(feat)
        if targets is not None:
            loss = self.loss_fn(x,targets)
            return x, loss
        
        return feat, x    #Extracted deep-features represented as 'feat'


def resnet152(pretrained=False, **kwargs):
    model = ResNet(Bottleneck, [3, 8, 36, 3], **kwargs)
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls['resnet152']))
    return model

In [6]:
def load_model(num_classes):
    model = resnet152(pretrained=True)
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs,num_classes) 
    return model

In [7]:
import torch
from torch.utils.data import Dataset
import numpy as np
import cv2
import albumentations as A

class CustomImageDataset(Dataset):
    def __init__(self, pixel_arrays, targets, resize=None, augmentations=None):
        self.pixel_arrays = pixel_arrays
        self.targets = targets
        self.resize = resize
        self.augmentations = augmentations

    def __len__(self):
        return len(self.pixel_arrays)

    def __getitem__(self, idx):
        # Convert the 1D array to a 2D grayscale image
        image = np.array(self.pixel_arrays[idx], dtype=np.float32).reshape(48, 48)
        
        # Resize image if needed
        if self.resize:
            image = cv2.resize(image, self.resize)

        # Expand dimensions to (48, 48, 1) and then convert to (new_size, new_size, 3) for RGB
        image = np.expand_dims(image, axis=-1)
        image = np.repeat(image, 3, axis=-1)  # Convert to RGB

        # Apply augmentations if specified
        if self.augmentations:
            augmented = self.augmentations(image=image)
            image = augmented['image']

        # Convert image to PyTorch tensor and permute to (3, new_size, new_size) format
        image = torch.tensor(image, dtype=torch.float32).permute(2, 0, 1)  # For RGB, shape becomes (3, new_size, new_size)
        target = torch.tensor(self.targets[idx], dtype=torch.long)

        return image, target
import torch
import numpy as np
from sklearn import metrics
from tqdm import tqdm
class Engine:
    @staticmethod
    def train(data_loader, model, optimizer, device, centerloss, nllloss, loss_weight=0.001, scheduler=None, accumulation_steps=1, fp16=False):
        model.train()
        losses = AverageMeter()
        scaler = torch.cuda.amp.GradScaler() if fp16 else None

        if accumulation_steps > 1:
            optimizer[0].zero_grad()  # Zero grad for model optimizer
            optimizer[1].zero_grad()  # Zero grad for center loss optimizer

        tk0 = tqdm(data_loader, total=len(data_loader))

        for batch_idx, (images, targets) in enumerate(tk0):
            images = images.to(device)
            targets = targets.to(device)

            optimizer[0].zero_grad()
            optimizer[1].zero_grad()

            with torch.set_grad_enabled(True):
                if fp16:
                    with torch.cuda.amp.autocast():
                        feats, outputs = model(images)
                        loss = centerloss(feats, targets) * loss_weight + nllloss(outputs, targets)
                    scaler.scale(loss).backward()
                else:
                    feats, outputs = model(images)
                    loss = centerloss(feats, targets)* loss_weight + nllloss(outputs, targets)
                    loss.backward()

                # Step the optimizers after accumulating gradients
                if (batch_idx + 1) % accumulation_steps == 0:
                    if fp16:
                        scaler.step(optimizer[0])
                        scaler.step(optimizer[1])
                        scaler.update()
                    else:
                        optimizer[0].step()  # Update model parameters
                        optimizer[1].step()  # Update center loss parameters

                    if scheduler:
                        scheduler.step()

                    optimizer[0].zero_grad()
                    optimizer[1].zero_grad()

            # Update average loss
            losses.update(loss.item(), data_loader.batch_size)
            tk0.set_postfix(loss=losses.avg)

        return losses.avg

    @staticmethod
    def evaluate(data_loader, model, device, use_tpu=False):
        losses = AverageMeter()
        final_predictions = []
        model.eval()
        with torch.no_grad():
            tk0 = tqdm(data_loader, total=len(data_loader), disable=use_tpu)
            for b_idx, data in enumerate(tk0):
                images, targets = data  # Adjust if your data format is different

                # Move tensors to device
                images = images.to(device)
                targets = targets.to(device)

                # Forward pass
                predictions, loss = model(images, targets)
                predictions = predictions.cpu()
                losses.update(loss.item(), images.size(0))
                final_predictions.append(predictions)
                tk0.set_postfix(loss=losses.avg)

        # Concatenate all predictions and convert to NumPy array
        final_predictions = torch.cat(final_predictions).numpy()
        return final_predictions, losses.avg
    def predict(data_loader, model, device, use_tpu=False):
        model.eval()
        final_predictions = []
        with torch.no_grad():
            tk0 = tqdm(data_loader, total=len(data_loader), disable=use_tpu)
            for b_idx, data in enumerate(tk0):
                inputs, _ = data  # Unpack data
                inputs = inputs.to(device)
                predictions = model(inputs) 
                predictions = F.softmax(predictions,dim = 1)# Assume model returns only predictions
                final_predictions.append(predictions.cpu())
                tk0.set_postfix()
        return torch.cat(final_predictions).numpy()
        # Concatenate all predictions and convert to numpy array
        return torch.cat(final_predictions).numpy()
class AverageMeter():
        def __init__(self):
            self.reset()

        def reset(self):
            self.val = 0
            self.avg = 0
            self.sum = 0
            self.count = 0

        def update(self, val, n=1):
            self.val = val
            self.sum += val * n
            self.count += n
            self.avg = self.sum / self.count

In [8]:
class CenterLoss(nn.Module):
    def __init__(self, num_classes=7, feat_dim=512, use_gpu=True):
        super(CenterLoss, self).__init__()
        self.num_classes = num_classes
        self.feat_dim = feat_dim
        self.use_gpu = use_gpu

        if self.use_gpu:
            self.centers = nn.Parameter(torch.randn(self.num_classes, self.feat_dim).cuda())
        else:
            self.centers = nn.Parameter(torch.randn(self.num_classes, self.feat_dim))

    def forward(self, x, labels):
        """
        Args:
            x: feature matrix with shape (batch_size, feat_dim).
            labels: ground truth labels with shape (batch_size).
        """
        batch_size = x.size(0)
        distmat = torch.pow(x, 2).sum(dim=1, keepdim=True).expand(batch_size, self.num_classes) + \
                  torch.pow(self.centers, 2).sum(dim=1, keepdim=True).expand(self.num_classes, batch_size).t()
        distmat.addmm_(1, -2, x, self.centers.t())

        classes = torch.arange(self.num_classes).long()
        if self.use_gpu: classes = classes.cuda()
        labels = labels.unsqueeze(1).expand(batch_size, self.num_classes)
        mask = labels.eq(classes.expand(batch_size, self.num_classes))

        dist = distmat * mask.float()
        loss = dist.clamp(min=1e-12, max=1e+12).sum() / batch_size

        return loss

In [9]:
from imblearn.over_sampling import SMOTE
def apply_smote_to_dataframe(df, pixel_column, label_column, random_state=42):
    """
    Apply SMOTE to a DataFrame containing pixel data in string format and class labels.
    
    Parameters:
    - df (pd.DataFrame): DataFrame containing the pixel data and labels.
    - pixel_column (str): Column name for the pixel strings.
    - label_column (str): Column name for the labels.
    - random_state (int): Seed for random number generation.
    
    Returns:
    - df_resampled (pd.DataFrame): DataFrame with resampled pixel data and labels.
    """
    # Extract pixel strings and labels from the DataFrame
    X_strings = df[pixel_column]
    y = df[label_column]

    # Convert the pixel strings into flattened arrays (2304 pixels per image)
    X_flattened = np.array([np.array(x.split(), dtype='float32') for x in X_strings])

    # Initialize SMOTE
    smote = SMOTE(random_state=random_state)

    # Apply SMOTE to the flattened arrays
    X_resampled_flattened, y_resampled = smote.fit_resample(X_flattened, y)

    # Convert the resampled arrays back into space-separated strings
    X_resampled_strings = [' '.join(map(str, x)) for x in X_resampled_flattened]

    # Create a new DataFrame with resampled pixel strings and labels
    df_resampled = pd.DataFrame({
        pixel_column: X_resampled_strings,
        label_column: y_resampled
    })

    return df_resampled

In [10]:
import albumentations as A
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
def train(fold):
    df = pd.read_csv(f"/kaggle/input/kiolpujy/class{fold}.csv")
    df_train, df_valid = train_test_split(df, test_size=0.2, random_state=42, stratify=df[f'class_{fold}'])
    pixel_column = 'pixels'
    label_column = f'class_{fold}'
    df_train = apply_smote_to_dataframe(df_train, pixel_column, label_column, random_state=42)
    df_train = df_train.reset_index(drop=True)
    df_valid = df_valid.reset_index(drop=True)
    device = "cuda"
    epochs = 50
    train_bs = 32
    valid_bs = 16
    train_pixels = df_train['pixels'].apply(lambda x: np.fromstring(x, sep=' ', dtype=np.float32))
    train_targets = df_train[f'class_{fold}'].values

    valid_pixels = df_valid['pixels'].apply(lambda x: np.fromstring(x, sep=' ', dtype=np.float32))
    valid_targets = df_valid[f'class_{fold}'].values

    mean = (0.485, 0.456, 0.406)
    std = (0.229, 0.224, 0.225)
    train_aug = A.Compose(
    [
        A.Resize(height=224, width=224),  # Resize images to 224x224
        A.Normalize(mean=mean, std=std, max_pixel_value=255.0, always_apply=True),
        A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.1, rotate_limit=15),
        A.HorizontalFlip(p=0.5)
    ]
    )

    valid_aug = A.Compose(
        [
        A.Resize(height=224, width=224),  # Resize images to 224x224
        A.Normalize(mean=mean, std=std, max_pixel_value=255.0, always_apply=True)
        ]
    )
    train_dataset = CustomImageDataset(
        pixel_arrays=train_pixels,
        targets=train_targets,
        resize=(224, 224),  # New size
        augmentations=train_aug,
    )

    train_loader = DataLoader(
        train_dataset, batch_size=32, shuffle=True, num_workers=4
    )

    valid_dataset = CustomImageDataset(
        pixel_arrays=valid_pixels,
        targets=valid_targets,
        resize=(224, 224),  # New size
        augmentations=valid_aug,
    )

    valid_loader = DataLoader(
        valid_dataset, batch_size=32, shuffle=False, num_workers=4
    )
    model = load_model(2)
    model.to(device)

    # Initialize CrossEntropyLoss and CenterLoss
    nllloss = nn.CrossEntropyLoss().cuda()
    centerloss = CenterLoss(num_classes=2, feat_dim=2048, use_gpu=True).cuda()
    loss_weight = 0.05 # Weight for the center loss

    # Initialize optimizers
    optimizer4nn = torch.optim.Adagrad(model.parameters(), lr=0.001)
    optimizer4center = torch.optim.Adagrad(centerloss.parameters(), lr=0.001)
    optimizer = [optimizer4nn, optimizer4center]

    # Create separate schedulers for both optimizers
    scheduler_nn = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer4nn, patience=3, threshold=0.001, mode="max"
    )
    
    scheduler_center = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer4center, patience=3, threshold=0.001, mode="max"
    )

    es = EarlyStopping(patience=5, mode="max")

    for epoch in range(epochs):
        # Train the model using the updated function
        train_loss = Engine.train(
            train_loader, model, optimizer, device, centerloss, nllloss, loss_weight
        )

        # Validation step
        predictions, valid_loss = Engine.evaluate(valid_loader, model, device=device)
        final_predictions = np.argmax(np.vstack(predictions), axis=1)
        valid_targets = np.array(valid_targets)

        accuracy = metrics.accuracy_score(valid_targets, final_predictions)
        print(f"Epoch = {epoch}, Accuracy = {accuracy:.4f}")

        # Step both schedulers based on the validation accuracy
        scheduler_nn.step(accuracy)
        scheduler_center.step(accuracy)

        # Early stopping
        es(accuracy, model, model_path=f"model_fold_RE152class{fold}.bin")
        if es.early_stop:
            print("Early stopping")
            break

    # Return OOF data
    oof_data = {
        'id': df_valid.index,
        'true_emotion': valid_targets,
        'pred_emotion': final_predictions
    }

    return pd.DataFrame(oof_data)

In [11]:
X = np.empty((0, 3))  

for i in range(0, 5):
    df = train(i)
    df.to_csv(f"oof1EFOCAL-{i}.csv", index=False, header=False)  
    X = np.concatenate((X, df.values))  
df = pd.DataFrame(X, columns=['id', 'true_emotion', 'pred_emotion']) 
df.to_csv('RES152B.csv', index=False)

Downloading: "https://download.pytorch.org/models/resnet152-b121ed2d.pth" to /root/.cache/torch/hub/checkpoints/resnet152-b121ed2d.pth
100%|██████████| 230M/230M [00:00<00:00, 294MB/s]
  0%|          | 0/215 [00:00<?, ?it/s]/tmp/ipykernel_24/3583249165.py:22: UserWarning: This overload of addmm_ is deprecated:
	addmm_(Number beta, Number alpha, Tensor mat1, Tensor mat2)
Consider using one of the following signatures instead:
	addmm_(Tensor mat1, Tensor mat2, *, Number beta = 1, Number alpha = 1) (Triggered internally at /usr/local/src/pytorch/torch/csrc/utils/python_arg_parser.cpp:1581.)
  distmat.addmm_(1, -2, x, self.centers.t())
100%|██████████| 32/32 [00:07<00:00,  4.16it/s, loss=0.478]


Epoch = 0, Accuracy = 0.7840
Validation score improved (-inf --> 0.784). Saving model!


100%|██████████| 32/32 [00:07<00:00,  4.17it/s, loss=0.417]


Epoch = 1, Accuracy = 0.8310
Validation score improved (0.784 --> 0.831). Saving model!


100%|██████████| 32/32 [00:07<00:00,  4.19it/s, loss=0.363]


...

Epoch = 6, Accuracy = 0.7790
EarlyStopping counter: 3 out of 5


100%|██████████| 32/32 [00:07<00:00,  4.08it/s, loss=0.55]


Epoch = 7, Accuracy = 0.8240
EarlyStopping counter: 4 out of 5


100%|██████████| 32/32 [00:07<00:00,  4.18it/s, loss=0.608]

Epoch = 8, Accuracy = 0.8100
EarlyStopping counter: 5 out of 5
Early stopping
